# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
wdf=pd.read_csv("/Users/yxair/Desktop/NW Bootbcamps/nu-chi-data-pt-02-2020-u-c-master/Homework/06-Python-APIs/Instructions/output_data/CityWeatherData.csv")
wdf.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
0,yerbogachen,100,RU,1586583228,79,61.28,108.01,36.66,7.34
1,ushuaia,40,AR,1586583229,75,-54.80,-68.30,41.00,4.70
2,alice springs,9,AU,1586583229,14,-23.70,133.88,84.20,10.29
3,hermanus,99,ZA,1586583229,91,-34.42,19.23,62.01,1.99
4,auckland,43,NZ,1586583002,59,-36.87,174.77,66.00,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = wdf[["Lat", "Lng"]]
humidity = wdf["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2.5)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal.


df_temp=wdf[(wdf["Max_Temp"]>70) & (wdf["Max_Temp"]<80)]
df_wsp=df_temp[df_temp["Wind_Speed"]<10]
df_cld=df_wsp[df_wsp["Cloudiness"]==0]
df_cln=df_cld.dropna()

df_cln.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max_Temp,Wind_Speed
129,coahuayana,0,MX,1586583245,45,18.73,-103.68,75.69,3.29
188,lazaro cardenas,0,MX,1586583271,85,17.96,-102.20,72.18,4.41
272,dharchula,0,IN,1586583281,15,29.85,80.53,76.35,5.23
273,tombouctou,0,ML,1586583281,12,20.00,-3.00,77.79,5.97
356,jogindarnagar,0,IN,1586583292,16,31.98,76.77,79.95,6.24


In [7]:
new_locations = df_cln[["Lat", "Lng"]]
new_humidity = df_cln["Humidity"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(new_locations, weights=new_humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = wdf[["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

hotel_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Lng,Hotel Name
0,yerbogachen,RU,61.28,108.01,
1,ushuaia,AR,-54.80,-68.30,
2,alice springs,AU,-23.70,133.88,
3,hermanus,ZA,-34.42,19.23,
4,auckland,NZ,-36.87,174.77,


# IT'S RUNNING OVER 15 MINS, STOPED IT AS AVOID TO CHARGE MILLIONS OF DOLLARS

In [9]:
hotel_name = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


for i, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response= requests.get(base_url, params=params) 
    response_json=response.json()
    try:
        hotel_name.append(response_json['results'][0]['name'])
    except Exception:
        pass

KeyboardInterrupt: 

In [ ]:
hotel_df["Hotel Name"] = hotel_name

hotel_df.head(5)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display Map